In [51]:
import numpy as np
from collections import OrderedDict

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [52]:
# Creating the chip
design = designs.DesignPlanar()
gui = MetalGUI(design)

# allowing overwrite
design.overwrite_enabled = True

# Construct

In [53]:
# importing the qubit
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

# importing the meander resonator
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [54]:
# assigning the grid size
n_x = 4
n_y = 3

In [55]:
# changing the chip size
design.chips.main.size.size_x = str((n_x+1)*3)+'mm'
design.chips.main.size.size_x = str((n_y+1)*3)+'mm'

design.chips.main.size.center_x = str((n_x-1)*1.5)+'mm'
design.chips.main.size.center_y = str((n_y-1)*1.5)+'mm'

In [ ]:
# to draw the qubits we will use two for loops
for x in range(n_x):
    for y in range(n_y):
        options = dict(
            pos_x=str(x*3000)+'um',
            pos_y=str(y*3000 + (x%2)*1500)+'um',
            orientation='-90',
            connection_pads= dict(
                b0 = dict(loc_W=-1, loc_H=-1, pad_width='75um'),
                b1 = dict(loc_W=-1, loc_H=+1, pad_width='120um'),
                b2 = dict(loc_W=+1, loc_H=-1, pad_width='120um'),
                b3 = dict(loc_W=+1, loc_H=+1, pad_width='90um')
            )
        )

        obj = TransmonPocket(design, 'Q'+str(x)+'_'+str(y), options)

gui.rebuild()
gui.autoscale()

In [ ]:
# connecting the qubits
for x in range(n_x):
    for y in range(n_y):
        if y<(n_y-1):
            options = dict(
                total_length=str(7+(y%2)*0.5)+'mm',
                fillet='99um',
                lead= dict(
                    start_straight='0.5mm',
                    end_straight='0.25mm'
                ),
                meander=dict(asymmetry='-700um'),
                pin_inputs=dict(
                    start_pin=dict(
                        component='Q'+str(x)+'_'+str(y),
                        pin='b0'
                    ),
                    end_pin=dict(
                        component='Q'+str(x)+'_'+str(y+1),
                        pin='b3'
                    )
                )
            )
            connectorAD = RouteMeander(design, 'CU'+str(x)+'_'+str(y),options)
        
        if x<(n_x-1) and (not(x%2 and y==(n_y-1))):
            options = dict(
                total_length=str(6+(y%2)*0.5)+'mm',
                fillet='99um',
                lead= dict(
                    start_straight='0.3mm',
                    end_straight='0.25mm'
                ),
                meander=dict(asymmetry='-200um'),
                pin_inputs=dict(
                    start_pin=dict(
                        component='Q'+str(x)+'_'+str(y),
                        pin='b1'
                    ),
                    end_pin=dict(
                        component='Q'+str(x+1)+'_'+str(y+(x%2)),
                        pin='b2'
                    )
                )
            )
            connectorBC = RouteMeander(design,'CS'+str(x)+'_'+str(y), options)

gui.rebuild()
gui.autoscale()

# Analyze

In [58]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, 'q3d')

In [ ]:
c1.sim.setup

In [ ]:
# Changing name of setup and increase passes
c1.sim.setup.name = "LOM"
c1.sim.setup.max_passes = 14

c1.sim.setup

In [ ]:
# simulating only one qubit
c1.sim.run(components=['Q0_0'], open_terminations=[('Q0_0', 'b0'), ('Q0_0', 'b1'), ('Q0_0', 'b2'), ('Q0_0', 'b3')])

In [ ]:
# we use the result of the simulation to complete the LOM analysis
c1.setup.junctions = Dict({'Lj': 12.31, 'Cj': 2})
c1.setup.freq_readout = 6.6
c1.setup.freq_bus = [6.0, 6.2,6.4]

c1.run_lom()

In [ ]:
# Finally we close the simulation
c1.sim.close()

# GDS

In [ ]:
chip_gds = design.renderers.gds
chip_gds.options

In [60]:
chip_gds.options['no_cheese']['buffer'] = '50um'
chip_gds.options['path_filename'] = '../../resources/Fake_Junctions.GDS'

In [ ]:
chip_gds.export_to_gds(f"{n_x}x{n_y}_Chip.gds")